# Prepare data for information retrieval fine-tuning 

## Map Bible verse to body segments with the relevant citations

In [1]:
import re,json,os
import pandas as pd 
from tqdm import tqdm 

In [7]:
def process(era,c_type="verse"): 
    relevant = {}
    with open(f"../assets/citations/{era}_citation_segments.json") as file:
        c_to_seg = json.load(file)
    seg_to_c = {}
    for cited, segments in c_to_seg.items():
        if "Ibidem" in cited: continue 
        cited = " ".join(cited.split("-"))
        if re.search(r"\d+ \d+",cited):
            cited = cited.split(" ")
            if c_type == "verse": 
                cited = " ".join(cited[:-2]) +" " + ".".join(cited[-2:])
            else: cited = " ".join(cited)
        for s in segments: 
            seg_id = (s.split(",")[0],int(s.split(",")[1]))
            if seg_id not in seg_to_c: 
                seg_to_c[seg_id] = []
            seg_to_c[seg_id].append(cited)
    print(era, len(c_to_seg),'citations',len(seg_to_c),"segments")

    for fp in tqdm(os.listdir(f"/Users/amycweng/DH/SERMONS_APP/db/data/{era}")):
        if "body" not in fp: continue 
        text = pd.read_csv(f"/Users/amycweng/DH/SERMONS_APP/db/data/{era}/{fp}", header=None)
        for idx, tcpID in enumerate(text[0]):
            sidx = str(text[1][idx])
            
            close_to_citation = False 
            citation_sidx = []
            # window of 2 segments 
            if (tcpID,int(sidx)) in seg_to_c:
                close_to_citation = True
                citation_sidx.append(int(sidx))
            else: 
                i = 1 
                if (tcpID,int(sidx)-i) in seg_to_c:
                    close_to_citation = True
                    citation_sidx.append(int(sidx)-i)
                elif (tcpID,int(sidx)+i) in seg_to_c:
                    close_to_citation = True 
                    citation_sidx.append(int(sidx)+i)
            if close_to_citation: 
                s = text[7][idx]
                t = text[6][idx]
                t = re.sub(r"\<\/i\>|\<NOTE\>|NONLATINALPHABET|\<i\>","",t)
                t = re.sub(r"\s+"," ",t)
                t = t.strip(" ")
                if len(t.split(" "))< 5: continue # at least 5 words long
                if s not in relevant: 
                    relevant[s] = ({},{},{}) # citations, original, location 
                relevant[s][2][(tcpID,sidx)] = None 
                relevant[s][1][t] = None 
                for entry in citation_sidx: 
                    for c in seg_to_c[(tcpID,entry)]:
                        relevant[s][0][c] = None 
    for s,r in relevant.items(): 
        relevant[s] = (list(r[0].keys()), list(r[1].keys()), list(r[2].keys()))
    print(len(set(relevant)),'unique passages')
    with open(f"../assets/relevant/{era}_citations.json","w+") as file: 
        json.dump(relevant, file)


In [8]:
with open('../assets/corpora.json',"r") as file: 
    eras = json.load(file)
for era in eras: 
    process(era)

pre-Elizabethan 1255 citations 5335 segments


100%|██████████| 19/19 [00:01<00:00, 10.75it/s]


13180 unique passages
Elizabethan 18612 citations 66304 segments


100%|██████████| 19/19 [00:18<00:00,  1.05it/s]


157078 unique passages
Jacobean 28028 citations 128122 segments


...

600644 unique passages
JamesII 11290 citations 17852 segments


100%|██████████| 28/28 [00:04<00:00,  6.97it/s]


42781 unique passages
WilliamAndMary 24357 citations 160023 segments


100%|██████████| 32/32 [00:24<00:00,  1.32it/s]


333484 unique passages


## Get non-citation segments 

In [13]:
# Body Texts  
from tqdm import tqdm 

def get_era(era):
    parts = {}
    for fp in tqdm(os.listdir(f"/Users/amycweng/DH/SERMONS_APP/db/data/{era}")):
        if "body" not in fp: continue 
        text = pd.read_csv(f"/Users/amycweng/DH/SERMONS_APP/db/data/{era}/{fp}", header=None)
        text = text.to_dict(orient="records")
        for item in text:
            original = item[6]
            original = re.sub(r"\<\/i\>|\<NOTE\>|NONLATINALPHABET|\<i\>","",original)
            original = re.sub(r"\s+"," ",original)
            original = original.strip(" ")
            tcpID, sidx = item[0], item[1]
            s = item[7]
            if not isinstance(s,str): continue 
            if len(s) == 0: continue 
            if s not in parts: 
                parts[s] = ({},{}) # original, locations 
            parts[s][1][(tcpID,str(sidx))] = None 
            parts[s][0][original] = None 
    for s,r in parts.items(): 
        parts[s] = [[str(k) for k in r[0]], list(r[1].keys())]

    with open(f"../assets/unique/{era}.json","w+") as file: 
        json.dump(parts,file)

    print(f"{era}: Total {len(parts)} parts.")

    # Marginalia 
    parts = {}
    for fp in tqdm(os.listdir(f"/Users/amycweng/DH/SERMONS_APP/db/data/{era}")):
        if "margin" not in fp: continue 
        margin = pd.read_csv(f"/Users/amycweng/DH/SERMONS_APP/db/data/{era}/{fp}", header=None, names=["tcpID","sidx","nidx","original","standardized"])
        margin = margin.to_dict(orient="records")
        for m in margin:
            t = m["original"]
            if isinstance(t,float): continue
            original = re.sub(r"\<\/i\>|\<NOTE\>|NONLATINALPHABET|\<i\>","",t)
            original = re.sub(r"\s+"," ",original)
            t = original.strip(" ")
            tcpID, sidx, nidx = m["tcpID"], m["sidx"], m["nidx"]
            s = m["standardized"]
            if not isinstance(s,str): continue 
            if len(s) == 0: continue 
            if s not in parts: 
                parts[s] = ({},{}) # original, locations 
            parts[s][1][(tcpID,str(sidx),str(nidx))] = None 
            parts[s][0][t] = None 
    for s,r in parts.items(): 
        parts[s] = [[str(k) for k in r[0]], list(r[1].keys())]
    with open(f"../assets/unique/{era}_margin.json","w+") as file: 
        json.dump(parts,file)
    print(f"{era} marginalia: Total {len(parts)} parts.")

In [10]:
get_era("pre-Elizabethan")

100%|██████████| 19/19 [00:02<00:00,  6.42it/s]


pre-Elizabethan: Total 109838 parts.


100%|██████████| 19/19 [00:00<00:00, 148.54it/s]


pre-Elizabethan marginalia: Total 7066 parts.


In [14]:
get_era("Elizabethan")

100%|██████████| 19/19 [00:29<00:00,  1.55s/it]


Elizabethan: Total 1038558 parts.


100%|██████████| 19/19 [00:01<00:00, 12.75it/s]


Elizabethan marginalia: Total 93532 parts.


In [15]:
get_era("CivilWar")

100%|██████████| 28/28 [00:18<00:00,  1.54it/s]


CivilWar: Total 719282 parts.


100%|██████████| 28/28 [00:00<00:00, 32.93it/s]


CivilWar marginalia: Total 50047 parts.


In [16]:
get_era("Jacobean")

100%|██████████| 22/22 [00:28<00:00,  1.27s/it]


Jacobean: Total 1114983 parts.


100%|██████████| 22/22 [00:02<00:00,  9.28it/s]


Jacobean marginalia: Total 109834 parts.


In [17]:
get_era("Carolinian")

100%|██████████| 33/33 [00:29<00:00,  1.13it/s]


Carolinian: Total 1167333 parts.


100%|██████████| 33/33 [00:01<00:00, 17.19it/s]


Carolinian marginalia: Total 96211 parts.


In [18]:
get_era("Interregnum")

 82%|████████▏ | 23/28 [00:21<00:05,  1.15s/it]/var/folders/3_/ylrg8wdj20l755p4921q0wg80000gp/T/ipykernel_54307/2248434086.py:8: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  text = pd.read_csv(f"/Users/amycweng/DH/SERMONS_APP/db/data/{era}/{fp}", header=None)
100%|██████████| 28/28 [00:31<00:00,  1.13s/it]


Interregnum: Total 1210755 parts.


100%|██████████| 28/28 [00:01<00:00, 20.16it/s]


Interregnum marginalia: Total 60876 parts.


In [19]:
get_era("JamesII")

100%|██████████| 28/28 [00:06<00:00,  4.66it/s]


JamesII: Total 230192 parts.


100%|██████████| 28/28 [00:00<00:00, 72.92it/s]


JamesII marginalia: Total 11589 parts.


In [20]:
get_era("CharlesII")

100%|██████████| 30/30 [01:07<00:00,  2.26s/it]


CharlesII: Total 2575232 parts.


100%|██████████| 30/30 [00:01<00:00, 16.40it/s]


CharlesII marginalia: Total 90319 parts.


In [21]:
get_era("WilliamAndMary")

100%|██████████| 32/32 [00:32<00:00,  1.02s/it]


WilliamAndMary: Total 1123151 parts.


100%|██████████| 32/32 [00:00<00:00, 70.48it/s]


WilliamAndMary marginalia: Total 25198 parts.
